In [ ]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

In [ ]:
# Choose the model
model_name = "tiiuae/falcon-7b-instruct"

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_name,use_auth_token=HUGGINGFACE_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Save model and tokenizer to Google Drive
# save_path = "../falcon_model"
# tokenizer.save_pretrained(save_path)
# model.save_pretrained(save_path)

# print(f"Model saved at {save_path}")

In [ ]:
# Define text-generation pipeline
from langchain_huggingface import HuggingFacePipeline


pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    max_new_tokens=256  # Limit response length
)

# Wrap pipeline for LangChain
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# JSON response format
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    }
}

# MCQ generation prompt
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, create a quiz with {number} multiple-choice questions
for {subject} students at a difficulty level of {tone}. Ensure questions follow the text accurately,
are not repeated, and match the RESPONSE_JSON format below.

### RESPONSE_JSON
{response_json}
"""


# Example input
TEXT = """Electromagnetic induction is the process of generating an electromotive force (EMF) or voltage across
a conductor when exposed to a changing magnetic field. This principle was discovered by Michael Faraday in 1831
and is the basis for electrical generators and transformers.

According to Faraday’s Law, the magnitude of the induced EMF is directly proportional to the rate of change
of magnetic flux through the conductor. Mathematically, it is expressed as:

𝓔 = -dΦB/dt

where 𝓔 is the induced EMF, and ΦB is the magnetic flux. The negative sign represents Lenz’s Law, which states
that the induced current will always oppose the change in magnetic flux that caused it.

There are two main types of electromagnetic induction: mutual induction and self-induction. Mutual induction
occurs when a changing current in one coil induces an EMF in a nearby coil, as seen in transformers. Self-induction
occurs when a changing current in a coil induces an EMF in the same coil due to its own changing magnetic field.

This principle has widespread applications, including power generation, wireless charging, and credit card readers."""

In [ ]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

NUMBER = 5
SUBJECT = "Electromagnetic Induction"
TONE = "simple"

# Generate MCQs
response = quiz_chain({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON)
})

# Print results
print("Generated MCQs:\n", response["quiz"])